In [87]:
import torch
import torch.nn.functional as F
from itertools import chain

In [23]:
padding = 3
with open('names.txt', 'r') as file:
    names = file.readlines()
names = ['.'*padding+name.strip()+'.' for name in names]
names[:3]

['...emma.', '...olivia.', '...ava.']

In [35]:
unique_char = set(list(chain(*names)))
length = len(unique_char)
chtoi = {i:en for en, i in enumerate(sorted(list(unique_char)))}
itoch = {v:k for k,v in chtoi.items()}
length

27

In [44]:
Xs, Ys = [], []
for word in names[:10]:
    for c1, c2, c3, c4 in zip(word, word[1:], word[2:], word[3:]):
        Xs.append([chtoi[c1],chtoi[c2],chtoi[c3]])
        Ys.append(chtoi[c4])
Xs[:10], Ys[:10]

([[0, 0, 0],
  [0, 0, 5],
  [0, 5, 13],
  [5, 13, 13],
  [13, 13, 1],
  [0, 0, 0],
  [0, 0, 15],
  [0, 15, 12],
  [15, 12, 9],
  [12, 9, 22]],
 [5, 13, 13, 1, 0, 15, 12, 9, 22, 9])

In [99]:
embedding_dim = 2
hid_dim = [padding*embedding_dim, 100]

lr = 1
embed = torch.rand((length, embedding_dim))
print(f'{embed[Xs].shape=}')

W1 = torch.randn(hid_dim[0],hid_dim[1], requires_grad=True)
b1 = torch.randn(hid_dim[1], requires_grad=True)
W2 = torch.randn(hid_dim[1],length, requires_grad=True)
b2 = torch.randn(length, requires_grad=True)
parameters = [W1, b1, W2, b2]
print(f'{W1.shape=}, {b1.shape=}, {W2.shape=}, {b2.shape=}')

for i in range(1000):
    hidden = torch.tanh(embed[Xs].view(-1, hid_dim[0]) @ W1 + b1)
    out = hidden @ W2 + b2
    loss = F.cross_entropy(out, torch.tensor(Ys)) 
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data -= lr * p.grad
print(f'loss={loss.item()}')

embed[Xs].shape=torch.Size([67, 3, 2])
W1.shape=torch.Size([6, 100]), b1.shape=torch.Size([100]), W2.shape=torch.Size([100, 27]), b2.shape=torch.Size([27])
loss=0.5360665321350098
